# Exp1. Activation-map visualization (AGNet vs. GuidelessNet)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys, os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="9"

sys.path.append(os.path.dirname(os.getcwd()))

In [3]:
from types import SimpleNamespace
import yaml
import utils.data
from utils.generate_model import load_trained_ckpt

In [4]:
# load configurations
conf_dict = yaml.load(open('../cfg/AGNet_URFD.yaml'), Loader=yaml.FullLoader)
conf_dict = {k: '' if v is None else v for k,v in conf_dict.items()}
opt = SimpleNamespace(**conf_dict)
opt

namespace(attention=False, backbone='r2plus1d_18', batch_size=64, ckpt_dir='/data/GaitData/ckpt_dir', data_root='/data/FallDownData/URFD/video', dataset='URFD', device_ids='0', enable_guide=True, img_size=144, input_file='../../../preprocess/data/person_detection_and_tracking_results_drop-URFD.pkl', mean_dataset='kinetics', merge_type='', mode='test', model_arch='AGNet-pretrain', model_depth=18, multi_gpu=True, n_groups=-1, n_threads=8, no_mean_norm=False, norm_value=255, num_units=256, precrop=False, pretrained_path='', sample_duration=16, sample_size=112, std_norm=False, test_fold=1, train_crop='', with_segmentation=True)

In [5]:
opt, net, criterion1, criterion2, optimizer, scheduler, spatial_transform, temporal_transform, target_transform, plotter, train_loader, test_loader, target_columns = utils.data.prepare_data(
    opt, opt.test_fold)
net = load_trained_ckpt(opt, net)

Setting up a new session...



Load trained model from /data/GaitData/ckpt_dir/NonAttentive_AGNet-pretrain_r2plus1d_18-18_URFD/model_fold-1.pth...


In [6]:
import torch
import numpy as np
import cv2
from utils.validate_activations import ActivationMapProvider

In [7]:
# init ActivationMapProvider
actmap_provider = ActivationMapProvider(net, opt)

In [8]:
def denormalization(img, mean, std):
    return img.permute(1, 2, 0).numpy()*std + mean

In [9]:
test_ds = test_loader.dataset
layer_sel = f"{opt.model_arch.split('-')[0].lower()}.conv_1x1"

import tqdm

i = 1500  # falling-sample index    
img_tensor, mask_tensor, target, vid, _ = test_ds[i]
print(target)
if torch.cuda.is_available():
    img_tensor = img_tensor.cuda()

activation_result = actmap_provider.compute(
    img_tensor=img_tensor, activation_layer=layer_sel)

# Move CUDA tensor to CPU
img_tensor = img_tensor.cpu()

overlay_seq = []

for t in range(img_tensor.size(1)):
    img_ = np.uint8(
        255*denormalization(img_tensor[:, t], opt.mean, opt.std))
    heatmap_ = np.uint8(255*activation_result[t])
    heatmap_ = cv2.applyColorMap(heatmap_, cv2.COLORMAP_JET)
    frame = np.uint8(heatmap_*0.3 + img_*0.5)
    overlay_seq.append(frame)

1


In [10]:
video_filename = f'falling-sample-{opt.model_indicator}.avi'
out = cv2.VideoWriter(video_filename,
                      cv2.VideoWriter_fourcc(*'DIVX'),
                      24.0, (opt.sample_size, opt.sample_size))
for frame in overlay_seq:
    out.write(frame)

out.release()

video_name = os.path.splitext(video_filename)[0]

In [11]:
!ffmpeg -i "{video_name}.avi" "{video_name}.mp4" -y -loglevel quiet

In [12]:
html_code = f"""
<video width="320" height="240" controls>
  <source src="{video_name}.mp4" type="video/mp4">
</video>
"""

from IPython.display import HTML
HTML(html_code)

In [ ]:
import pandas as pd
detection_data = pd.read_pickle('../../../preprocess/data/person_detection_and_tracking_results_drop-URFD.pkl')
detection_data[detection_data.vids==vid].head()

In [ ]:
train_list = pd.read_csv('/data/FallDownData/URFD/TrainTestlist/trainlist01.txt',sep=' ', names=['vids', 'cls']).vids
import collections
d = collections.defaultdict(list)
for v in train_list:
    d[v[-8:]].append(v)
{ k : len(d[k]) for k in d.keys() }

In [ ]:
train_list = pd.read_csv('/data/FallDownData/URFD/TrainTestlist/trainlist01.txt',sep=' ', names=['vids', 'cls']).vids
import collections
d = collections.defaultdict(list)
for v in train_list:
    d[v.split('/')[0]].append(v)
{ k : len(d[k]) for k in d.keys() }

In [ ]:
train_list = pd.read_csv('/data/FallDownData/URFD/TrainTestlist/testlist01.txt',sep=' ', names=['vids', 'cls']).vids
import collections
d = collections.defaultdict(list)
for v in train_list:
    d[v[-8:]].append(v)
{ k : len(d[k]) for k in d.keys() }

In [ ]:
train_list = pd.read_csv('/data/FallDownData/URFD/TrainTestlist/testlist01.txt',sep=' ', names=['vids', 'cls']).vids
import collections
d = collections.defaultdict(list)
for v in train_list:
    d[v.split('/')[0]].append(v)
{ k : len(d[k]) for k in d.keys() }

In [ ]:
with open('/data/FallDownData/URFD/TrainTestlist/trainlist01.txt') as f:
    lines = [ line.strip() for line in f.readlines() ]
lines

# Exp 2 : performance metrices (classification_report, AUROC, ...)

In [ ]:
full_grp_names['fold'].append

In [ ]:
import h5py
import collections
f = h5py.File('../results.hdf5')
full_grp_names = collections.defaultdict(list)

def group_cb(name):
    if name.endswith('_vals'):
        name_splits = name.split('/')
        fold = name_splits[1]
        full_grp_names[fold].append('/'.join(name_splits[2:]))

f.visit(group_cb)
full_grp_names



In [ ]:
f['MulticamFD_NonAttentive_AGNet-pretrain_r2plus1d_18-18']['fold-9']['test']['epoch-1']['true_vals'].value

# Exp 3 : YOLO 의 오검출 케이스에 대한 AGNet 의 활성화맵 확인
- YOLO(C++) converting to pytorch model
- how to find false detection?
    1) false alarm : FP
    2) missing : FN
- missing case에서의 YOLO 의 사람에 대한 activation과 AGNet의 activation 비교?
    => YOLO가 잘 찾지 못하는 부분을 AGNet은 따라갈 수 있더라...
    